In [4]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
import io
from IPython.display import display

In [5]:
DATA = pd.read_csv('area_of_work_prediction3.csv')
DATA.head()

,domain,keywords,company_name,not_mapped,mapping_area,prediction
0,https://3pbio.com,"['recombinant proteins', 'clinical manufacturi...",3pbio,[],"{'recombinant proteins': ['DS'], 'clinical man...","{'DP': 0.2051, 'DS': 0.7436, 'AD': 0.0513}"
1,https://abzena.com,"['cell line development', 'mammalian cell cult...",Abzena,[],"{'cell line development': ['DS'], 'mammalian c...","{'DP': 0.0667, 'DS': 0.9333, 'AD': 0.0}"
2,https://adarepharmasolutions.com,"['drug product development', 'clinical trial',...",Adarepharmasolutions,[],"{'drug product development': ['DS'], 'clinical...","{'DP': 0.4556, 'DS': 0.4556, 'AD': 0.0889}"
3,https://adelphi-hp.com,"['vials', 'shelf life', 'syringes', 'vial', 'd...",Adelphi-hp,[],"{'vials': ['DP'], 'shelf life': ['DS'], 'syrin...","{'DP': 0.5556, 'DS': 0.3889, 'AD': 0.0556}"
4,http://alttox.org,"['research and development', 'cell lines', 'ph...",Alttox,[],"{'research and development': ['DS'], 'cell lin...","{'DP': 0.1667, 'DS': 0.7, 'AD': 0.1333}"


# Prediction widget

In [6]:
out = widgets.Output()#(layout={'border': '1px solid black'})

In [7]:
wlist = widgets.SelectMultiple(
    options=[],
    value=[],
    rows=10,
    description='Keywords',
    disabled=False
)

w_not_mapped_keywords = widgets.SelectMultiple(
    options=[],
    value=[],
    rows=10,
    description='Not mapped keywords',
    disabled=False
)

In [8]:
def test_keyword_match(row, keyword):
    _keywords = eval(row['keywords'])
    return any([keyword in _keyword for _keyword in _keywords])

In [9]:
#DATA.apply(test_keyword_match, axis=1, args=('resear', ))

In [10]:
def show_domain_():
    Domain = w_domains_list.value
    sub = DATA['domain'] == Domain
    if sum(sub) > 0:
        predictions = eval(DATA.loc[sub, 'prediction'].to_list()[0])
        kwords = eval(DATA.loc[sub, 'keywords'].to_list()[0])
        wlist.options = kwords
        name = DATA.loc[sub, 'company_name'].to_list()[0]
        text = list(predictions.keys())
        values = list(predictions.values())
        pred = text[values.index(max(values))]
        with out:
            out.clear_output()
            print('Company name: ', name)
            print('Predicted activity: ', pred.upper())
            plt.cla() # Clearing the ax
            plt.bar(list(range(len(values))), values, edgecolor='k')
            for idx, t in enumerate(text):
                plt.text(idx, values[idx], f'{100*values[idx]:.0f}%', va='bottom', ha='center', fontsize = 16)
            plt.xticks(list(range(len(values))), [s.upper() for s in text], fontsize=16)
            if max(values) < 0.9:
                plt.ylim(0, 1)
            else:
                plt.ylim(0, 1.2)
            plt.yticks([])
            plt.show('off')
    else:
        clear_all()
            
btn_approve = widgets.Button(
    description='Approve domain',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Approve!',
    layout=widgets.Layout(width='70%'),
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
btn_reject = widgets.Button(
    description='To blacklist',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Reject!',
    layout=widgets.Layout(width='70%'),
    icon='ban' # (FontAwesome names without the `fa-` prefix)
)
def approve(*args):
    pass
def reject(*args):
    pass
btn_approve.on_click(approve)
btn_reject.on_click(reject)

In [11]:
def filter_by_keyword(*args):
    clear_all()
    filter_keyword = w_filter_text.value
    if len(filter_keyword) >= 3:
        subset = DATA.apply(test_keyword_match, axis=1, args=(filter_keyword, ))
        filtered_domains = DATA[subset]['domain']
        w_domains_list.options = filtered_domains
    else:
        w_domains_list.options = DATA['domain'].to_list()

In [12]:
w_filter_text = widgets.Text(
    value='',
    placeholder='filter keywords...',
    description='Filter',
    disabled=False
)
btn_filter = button = widgets.Button(
    description='Filter!',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to filter',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
btn_filter.on_click(filter_by_keyword)
w_filter = widgets.HBox([w_filter_text, btn_filter])

In [13]:
domains_list = DATA['domain'].to_list()

In [14]:
w_domains_list = widgets.Dropdown(
    options=domains_list,
    value=domains_list[0],
    description='Domain'
)
def on_change(change):
    if change['type'] == 'change':
        show_domain_()
w_domains_list.observe(on_change)

In [15]:
def clear_all():
    wlist.options = []
    with out:
        out.clear_output()

In [16]:
rLayout = widgets.Layout(align_items='flex-end')

In [17]:
w_predictions = widgets.VBox([w_filter, widgets.HBox([widgets.VBox([w_domains_list, wlist, btn_approve, btn_reject], layout=rLayout), out])])

In [18]:
#w_predictions = widgets.VBox([w_filter, w, widgets.HBox([out, widgets.VBox([wlist, widgets.HBox([btn_approve, btn_reject])])])])

In [19]:
w_approved_domains = widgets.HBox([widgets.VBox([w_domains_list, wlist, btn_reject], layout=rLayout), out])
w_blacklisted_domains = widgets.HBox([widgets.VBox([w_domains_list, wlist, btn_approve], layout=rLayout), out])

In [20]:
# manual parse of the string into dictionary
text = DATA.iloc[0]['prediction']
tmp = {}
for s in text.replace('}', '').split(','):
    key = s.split("'")[1]
    val = float(s.split(':')[1].strip())
    tmp[key] = val
tmp

{'DP': 0.2051, 'DS': 0.7436, 'AD': 0.0513}

# File uploader widget

In [21]:
btn_file_uploader = widgets.FileUpload(
    accept='.txt,.csv',
    multiple=False
)
out_file = widgets.Output()
btn_file_upload = widgets.Button(
    description='Upload file!',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='upload',
    layout=widgets.Layout(width='300px'),
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
def upload_file(*args):
    uploaded = list(btn_file_uploader.value.keys())
    if len(uploaded) > 0:
        key = uploaded[0]
        try:
            tmp = pd.read_csv(io.BytesIO(btn_file_uploader.value[key]['content']))
            with out_file:
                out_file.clear_output()
                display(tmp)
        except:
            with out_file:
                out_file.clear_output()
                print('File type not supported yet')
btn_file_upload.on_click(upload_file)

In [22]:
w_file_uploader = widgets.VBox([widgets.HBox([btn_file_uploader, btn_file_upload]), out_file])

# Search engine widget

In [23]:
known_keywords = pd.read_csv('known_keywords.csv')['keyword'].to_list()

In [24]:
#known_keywords

In [25]:
btn_start_engine = widgets.Button(
    description='Run search',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run!',
    layout=widgets.Layout(width='300px', height='50px'),
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
w_known_keywords = widgets.SelectMultiple(
    options = known_keywords,
    description='Keywords',
    layout=widgets.Layout(width='300px'),
    rows=10
        
)
btn_remove_keyword = widgets.Button(
    description='Remove selected',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Remove',
    layout=widgets.Layout(width='300px'),
    icon='ban' # (FontAwesome names without the `fa-` prefix)
)
btn_add_keyword = widgets.Button(
    description='Add new',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Add',
    icon='plus' # (FontAwesome names without the `fa-` prefix)
)
txt_add_keyword = widgets.Text()
w_add_keyword = widgets.HBox([txt_add_keyword, btn_add_keyword], layout=widgets.Layout(width='300px'))

def remove_keyword(*args):
    global known_keywords
    selected = w_known_keywords.value
    if len(selected) > 0:
        known_keywords = [kw for kw in known_keywords if kw not in selected]
        w_known_keywords.options = known_keywords

def add_keyword(*args):
    global known_keywords
    kw = txt_add_keyword.value
    if len(kw) >= 3:
        if kw not in known_keywords:
            known_keywords.append(kw)
            w_known_keywords.options = known_keywords
        
btn_add_keyword.on_click(add_keyword)
btn_remove_keyword.on_click(remove_keyword)

In [26]:
w_search_engine = widgets.VBox([btn_start_engine, w_known_keywords, btn_remove_keyword, w_add_keyword])

# KEYWORDS widget

In [30]:
txt_add_keyword_ = widgets.Text(
    layout=widgets.Layout(width='150px'),
    placeholder='new keyword...',
)
txt_keyword_category_ = widgets.Dropdown(
    options=['DP', 'DS', 'AD'],
    layout=widgets.Layout(width='150px'),
    value='DP',
    description='',
)
btn_add_keyword_ = widgets.Button(
    description='Add new',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Add',
    layout=widgets.Layout(width='150px'),
    icon='plus' # (FontAwesome names without the `fa-` prefix)
)
btn_remove_keyword_ = widgets.Button(
    description='Remove selected',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Remove',
    layout=widgets.Layout(width='150px'),
    icon='ban' # (FontAwesome names without the `fa-` prefix)
)
list_keywords_ = widgets.Select(
    options = known_keywords[:20],
    rows=15,
    layout=widgets.Layout(width='250px')
)
def choose_category(change):
    global known_keywords
    if change['type'] == 'change':
        value = txt_keyword_category_.value
        if value == 'DP':
            list_keywords_.options = known_keywords[:20]
        elif value == 'AD':
            list_keywords_.options = known_keywords[20:40]
        elif value == 'DS':
            list_keywords_.options = known_keywords[40:60]

txt_keyword_category_.observe(choose_category)
w_domains_list.observe(on_change)

In [31]:
w_add_keyword_ = widgets.HBox([txt_add_keyword_, btn_add_keyword_])
w_add_keyword_

In [33]:
w_keywords = widgets.HBox([widgets.VBox([txt_keyword_category_, txt_add_keyword_, btn_add_keyword_, btn_remove_keyword_]), list_keywords_])
#w_keywords

# MAIN WIDGET

In [34]:
children = [w_search_engine, w_predictions, w_approved_domains, w_blacklisted_domains, w_keywords, w_file_uploader]
w_main = widgets.Tab()
w_main.children = children
#tab.titles = ('New domains', ) # does not work, wtf
w_main.set_title(0, 'Search engine')
w_main.set_title(1, 'New domains')
w_main.set_title(2, 'Approved domains')
w_main.set_title(3, 'Domains blacklist')
w_main.set_title(4, 'Keywords')
w_main.set_title(5, 'Upload file')

In [35]:
w_main